# 时间组合特征

In [24]:
import os, codecs
import pandas as pd
import numpy as np
import calendar
from datetime import datetime

In [17]:
file_path = "../data/online retail-utf-8.csv"

In [18]:
df_scanner = pd.read_csv("../data/scanner_data.csv")

In [19]:
df_scanner.head()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88


## 标签内容
- Date日期
- Customer_ID 客户ID
- Transaction_ID 交易ID 
- SKU_Category SKU类别
- SKU SKU
- Quantity  数量
- Sales_Amount 销售金额

## 特征工程--无序类别&时间信息的组合特征！

[本内容来源微信公众号：kaggle竞赛宝典](https://mp.weixin.qq.com/s/uluU5DE_DJAWjf-yc4COiQ)


介绍10大与时间相关的组合特征，这些特征在95%涉及到时间信息的竞赛中都是极为重要的特征，也是以往Top选手与新手拉开差距的重要部分。

## 基本面

In [20]:
df_scanner.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131706 entries, 0 to 131705
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      131706 non-null  int64  
 1   Date            131706 non-null  object 
 2   Customer_ID     131706 non-null  int64  
 3   Transaction_ID  131706 non-null  int64  
 4   SKU_Category    131706 non-null  object 
 5   SKU             131706 non-null  object 
 6   Quantity        131706 non-null  float64
 7   Sales_Amount    131706 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 8.0+ MB


In [21]:
df_scanner.describe()

,Unnamed: 0,Customer_ID,Transaction_ID,Quantity,Sales_Amount
count,131706.000000,131706.000000,131706.000000,131706.000000,131706.000000
mean,65853.500000,12386.450367,32389.604187,1.485311,11.981524
std,38020.391614,6086.447552,18709.901238,3.872667,19.359699
min,1.000000,1.000000,1.000000,0.010000,0.020000
25%,32927.250000,7349.000000,16134.000000,1.000000,4.230000
50%,65853.500000,13496.000000,32620.000000,1.000000,6.920000
75%,98779.750000,17306.000000,48548.000000,1.000000,12.330000
max,131706.000000,22625.000000,64682.000000,400.000000,707.730000


## 日期时间的基本处理

从上面的结果可以看出，“季节”，“假日”，“工作日”和“天气”列应为“分类”数据类型。但是，这些列的当前数据类型为“ int”。 让我们以以下方式转换数据集，以便我们可以开始使用我们的。

In [22]:
df_scanner['Date_ch'] =  df_scanner.Date.apply(lambda x: x.split("/")[2]+"-"+x.split("/")[1]+"-"+x.split("/")[0])

In [23]:
df_scanner['Date_ch']

0         2016-01-02
1         2016-01-02
2         2016-01-02
3         2016-01-02
4         2016-01-02
             ...    
131701    2016-07-04
131702    2016-07-04
131703    2016-07-04
131704    2016-07-04
131705    2016-07-04
Name: Date_ch, Length: 131706, dtype: object

In [25]:
## 小时提取
# df_scanner["hour"] = df_scanner.datetime.apply(lambda x : x.split()[1].split(":")[0])
## 星期提取
df_scanner['weekday'] = df_scanner.Date_ch.apply(lambda dateString : calendar.day_name[datetime.strptime(dateString, "%Y-%m-%d").weekday()])
df_scanner['month'] = df_scanner.Date_ch.apply(lambda dateString : calendar.month_name[datetime.strptime(dateString,"%Y-%m-%d").month])

In [26]:
df_scanner.head()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Date_ch,weekday,month
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13,2016-01-02,Saturday,January
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46,2016-01-02,Saturday,January
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35,2016-01-02,Saturday,January
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59,2016-01-02,Saturday,January
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88,2016-01-02,Saturday,January


In [33]:
## 将日期处理成可以加减的日期 
## 方法二
# dtime = pd.to_datetime(df['故障开始'])
# v = (dtime.values - np.datetime64('1970-01-01T08:00:00Z')) / np.timedelta64(1, 'ms')
# df['start_time'] = v
df_scanner["Date_timestamp"] = pd.to_datetime(df_scanner["Date_ch"]) 
print(df_scanner.head(5))
## timestamp 转 str 
## 方法二
## df['故障开始'] = pd.to_datetime(df['start_time'],unit='ms',origin=pd.Timestamp('1970-01-01 08:00:00'))
df_scanner["Date_timestamp"].apply(lambda s: datetime.strftime(s, "%Y-%m-%d"))

   Unnamed: 0        Date  Customer_ID  Transaction_ID SKU_Category    SKU  \
0           1  02/01/2016         2547               1          X52  0EM7L   
1           2  02/01/2016          822               2          2ML  68BRQ   
2           3  02/01/2016         3686               3          0H2  CZUZX   
3           4  02/01/2016         3719               4          0H2  549KK   
4           5  02/01/2016         9200               5          0H2  K8EHH   

   Quantity  Sales_Amount     Date_ch   weekday    month Date_timestamp  
0       1.0          3.13  2016-01-02  Saturday  January     2016-01-02  
1       1.0          5.46  2016-01-02  Saturday  January     2016-01-02  
2       1.0          6.35  2016-01-02  Saturday  January     2016-01-02  
3       1.0          5.59  2016-01-02  Saturday  January     2016-01-02  
4       1.0          6.88  2016-01-02  Saturday  January     2016-01-02  


0         2016-01-02
1         2016-01-02
2         2016-01-02
3         2016-01-02
4         2016-01-02
             ...    
131701    2016-07-04
131702    2016-07-04
131703    2016-07-04
131704    2016-07-04
131705    2016-07-04
Name: Date_timestamp, Length: 131706, dtype: object

## 无序类别特征+时间特征 
无序分类变量是指所分类别或属性之间无程度和顺序的差别。                                                                                  
有序分类变量是指各类别之间有程度的差别。                           

为了方便表示我们将A作为无序类别特征，B作为时间特征。无序类别特征与时间特征的交互往往是决定比赛走向的一大极为重要的特征，此处我们就列举在实践过程中上分最多的几大特征。

## 时间进阶可处理
### 基于A/B一次差值特征
该特征可以表示A出现的时间间隔；该特征在点击预估等的问题中都是非常重要的特征。如果A表示商品的话，那么基于A/B差值特征就表示A商品被浏览/购买/点击的时间差。

In [35]:
df_scanner['SKU_Category_date_diff1'] = df_scanner['Date_timestamp'] - df_scanner.groupby('SKU_Category')['Date_timestamp'].shift()
df_scanner.head(5)

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Date_ch,weekday,month,Date_timestamp,SKU_Category_date_diff1
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13,2016-01-02,Saturday,January,2016-01-02,NaT
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46,2016-01-02,Saturday,January,2016-01-02,NaT
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35,2016-01-02,Saturday,January,2016-01-02,NaT
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59,2016-01-02,Saturday,January,2016-01-02,0 days
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88,2016-01-02,Saturday,January,2016-01-02,0 days


### 基于A/B二次差值特征
基于A/B一次差值特征的差值可以认为是**最近的(即shift(1)的)的差值**，自然我们也可以**是两次的，即shift(2)的**，或者多次的，但是实践中我们发现一般shift(3)及以上的效果就不是很明显了。

In [36]:
df_scanner['SKU_Category_date_diff2'] = df_scanner['Date_timestamp'] - df_scanner.groupby('SKU_Category')['Date_timestamp'].shift(2)
df_scanner.head(5)

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Date_ch,weekday,month,Date_timestamp,SKU_Category_date_diff1,SKU_Category_date_diff2
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13,2016-01-02,Saturday,January,2016-01-02,NaT,NaT
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46,2016-01-02,Saturday,January,2016-01-02,NaT,NaT
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35,2016-01-02,Saturday,January,2016-01-02,NaT,NaT
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59,2016-01-02,Saturday,January,2016-01-02,0 days,NaT
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88,2016-01-02,Saturday,January,2016-01-02,0 days,0 days


### 基于A与A/B一/二次差值特征的统计特征
该特征的构建是基于A/B一次/二次差值特征的，我们在A/B差值特征的基础之上再计算A关于A/B差值特征的统计特征，例如：
- 均值；
- 方差；
- 中位数；
- 偏度；
- 分位数(四分位数等)；
- 众数;
- skewness；
- 峰度系数；
- 其它统计特征。

```python
df['A_A_B_diff1_sts'] = df.groupby('A')['A_B_diff1'].apply(sts) 
df['A_A_B_diff2_sts'] = df.groupby('A')['A_B_diff2'].apply(sts) 
```

该特征可以很好地反映A在时间戳上的诸多信息，如果基于A与A/B差值特征的最大值和最小值的差值很小或者方差相对很小，这就暗示可能存在较强的周期性，A可能会出现在固定的时间周期内。

In [ ]:
df_scanner.groupby('SKU_Category')['SKU_Category_date_diff1'].apply(lambda x: print(x.mean()))

In [50]:
df_scanner['SKU_Category_SKU_Category_date_diff1_sts'] = df_scanner.groupby('SKU_Category')['SKU_Category_date_diff1'].apply(lambda x: x.mean())
# df_scanner['SKU_Category_SKU_Category_date_diff1_sts']

0 days 02:00:50.069541029
0 days 20:25:23.076923076
0 days 00:58:53.585241164
0 days 09:34:07.058823529
0 days 06:32:08.571428571
3 days 22:07:03.529411764
7 days 00:00:00
0 days 03:54:12.546916890
0 days 06:56:39.682034976
-1 days +12:42:21.176470589
0 days 06:03:27.407407407
0 days 02:21:55.050883770
16 days 05:03:09.473684210
0 days 01:27:23.271767810
NaT
5 days 00:00:00
0 days 20:34:17.142857142
0 days 02:33:46.697620429
0 days 10:35:23.741007194
24 days 13:05:27.272727272
0 days 11:12:14.693877551
0 days 20:09:03.396226415
1 days 19:12:00
1 days 16:00:00
1 days 02:07:03.529411764
1 days 09:08:34.285714285
1 days 05:31:30.647482014
35 days 00:00:00
1 days 18:15:23.076923076
0 days 05:10:17.142857142
0 days 02:18:02.451545311
7 days 03:33:20
11 days 04:38:42.580645161
0 days 09:41:03.157894736
18 days 00:00:00
12 days 07:40:48
8 days 08:43:38.181818181
0 days 15:53:05.611510791
0 days 01:42:01.524836349
32 days 08:00:00
1 days 23:44:00
0 days 02:58:54.368669817
1 days 09:04:51.89189

### 基于A/B差值的差值以及对应统计特征
如果说基于A/B差值特征是一阶滑动，那么基于A/B差值的差值就是二阶了，该特征一般直接加入也可以为模型带来微弱的提升。因为和基于A/B的差值是类似的，此处我们给出其计算方式，其它的不再赘述。

类似地，我们也可以做shift(N)，N>1的差值，但这么做的物理意义会差一些，实际中也很少能带来提升。

In [ ]:
df['A_B_diff1_diff'] = df['A_B_diff1'] - df.groupby('A')['A_B_diff1'].shift() 
df['A_B_diff1_diff_sts'] = df.groupby('A')['A_B_diff1_diff'].apply(sts) 

### A的第一次出现时间
A的第一次出现时间也就是接触A的第一次时间，该特征具有非常强的意义。


In [48]:
df_scanner['SKU_Category_date_first'] =  df_scanner.groupby('SKU_Category')['Date_timestamp'].first()  
df_scanner['SKU_Category_date_first']

0        NaT
1        NaT
2        NaT
3        NaT
4        NaT
          ..
131701   NaT
131702   NaT
131703   NaT
131704   NaT
131705   NaT
Name: SKU_Category_date_first, Length: 131706, dtype: datetime64[ns]

In [47]:
df_scanner.groupby('SKU_Category')['Date_timestamp'].first() 

SKU_Category
01F   2016-01-04
06Z   2016-01-08
0H2   2016-01-02
0KX   2016-01-02
0WT   2016-01-02
         ...    
Z4O   2016-01-02
Z99   2016-01-07
ZJG   2016-03-03
ZX5   2016-01-15
ZYU   2016-01-08
Name: Date_timestamp, Length: 187, dtype: datetime64[ns]

### A的最后一次出现时间
A的最后一次出现时间也就是接触A的最后一次时间，该特征和第一次时间互为补充。



In [51]:
## df['A_B_last'] =  df.groupby('A')['B'].last()
df_scanner.groupby('SKU_Category')['Date_timestamp'].last()  

SKU_Category
01F   2016-07-03
06Z   2016-07-03
0H2   2016-07-04
0KX   2016-07-03
0WT   2016-07-03
         ...    
Z4O   2016-07-04
Z99   2016-07-27
ZJG   2016-12-13
ZX5   2016-07-02
ZYU   2016-12-28
Name: Date_timestamp, Length: 187, dtype: datetime64[ns]

### 当前距离A出现第一次时间的时间差
A从第一次被接触到最后一次被接触的时间的差值。

In [52]:
## df['B_A_B_first_diff'] =  df['B'] - df['A_B_first']

### 当前距离A最后一次出现时间的时间差
A距离最后一次接触的时间的差值

In [53]:
# df['B_A_B_first_last'] =  df['B'] - df['A_B_last']

### A出现的时间gap
A最后一次出现和第一次出现的时间的差值。

In [54]:
## df['A_B_gap'] = df['A_B_last'] - df['A_B_first']

### 平均每次A出现的时间
A每次出现的平均时间。

In [ ]:
### df['A_B_first_last_div_count'] = (df['A_B_last'] - df['A_B_first']) / df['A_B_count'] 